In [1]:
!pip install -qqq datasets accelerate bitsandbytes peft transformers evaluate trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.6/504.6 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
import time
import evaluate
import pandas as pd
import numpy as np
from datasets import Dataset, load_dataset
import random

huggingface_dataset_name = "nihiluis/financial-advisor-100"

dataset = load_dataset(huggingface_dataset_name)

dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/539 [00:00<?, ?B/s]

(…)-00000-of-00001-f0708e72202ddcaa.parquet:   0%|          | 0.00/321k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answer', 'text'],
        num_rows: 100
    })
})

In [3]:
def format_instruction(question: str, answer: str):
  instruction_template = """
### Instruction:
Task: Provide detailed answers corresponding to the given questions.
Topic: {question}

### Guidelines:
1. Contextual Understanding: Ensure your answers are relevant and based on the context of the question.
2. Clarity and Detail: Elaborate on your responses to provide comprehensive explanations.
3. Accuracy: Verify the accuracy of your answers before submission.
4. Language Quality: Maintain a clear and concise writing style, free of grammatical errors.

Example:

### Question:
{question}

### Answer:
{answer}
""".strip()

def generate_instruction_dataset(data_point):

  return {
    "question": data_point["question"],
    "answer": data_point["answer"],
    "text": format_instruction(data_point["question"],data_point["answer"])}

def process_dataset(data: dataset):
  return (
    data.shuffle(seed=42)
    .map(generate_instruction_dataset).remove_columns(['id'])
  )

In [4]:
dataset["train"] = process_dataset(dataset["train"])

train_data = dataset['train'].shuffle(seed=42).select([i for i in range(80)]) #Depends on the size of dataset

test_data = dataset['train'].shuffle(seed=42).select([i for i in range(80,100)]) #80-20 split


train_data,test_data

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

(Dataset({
     features: ['question', 'answer', 'text'],
     num_rows: 80
 }),
 Dataset({
     features: ['question', 'answer', 'text'],
     num_rows: 20
 }))

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_id =  "togethercomputer/Llama-2-7B-32K-Instruct"
# model_id = "meta-llama/Llama-2-13b-chat-hf"
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

pytorch_model.bin.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

In [7]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():

     all_param += param.numel()
     if param.requires_grad:
         trainable_params += param.numel()
  print(
     f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100*trainable_params/ all_param}"
  )

In [8]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
   r=16,
   lora_alpha=64,
   # target_modules=["query_key_value"],
   target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], #specific to Llama models.
   lora_dropout=0.1,
   bias="none",
   task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
print_trainable_parameters(model)

trainable params: 16777216 || all params: 3517190144 || trainable%: 0.477006226934315


In [16]:
from transformers import TrainingArguments

OUTPUT_DIR = "llama2-financial-advisor"

training_arguments = TrainingArguments(
   per_device_train_batch_size=4,
   gradient_accumulation_steps=4,
   optim="paged_adamw_32bit",
   logging_steps=1,
   learning_rate=1e-4,
   fp16=True,
   max_grad_norm=0.3,
   num_train_epochs=2,
  #  evaluation_strategy="steps",
   eval_steps=0.2,
   warmup_ratio=0.05,
   save_strategy="epoch",
   group_by_length=True,
   output_dir=OUTPUT_DIR,
   report_to="tensorboard",
   save_safetensors=True,
   lr_scheduler_type="cosine",
   seed=42,
)

model.config.use_cache = True

In [21]:
from trl import SFTTrainer
trainer = SFTTrainer(
   model=model,
   train_dataset=train_data,
   eval_dataset=test_data,
   peft_config=lora_config,
   dataset_text_field="text",
   max_seq_length=1024,
   tokenizer=tokenizer,
   args=training_arguments
)

trainer.train()
from huggingface_hub import notebook_login

notebook_login()

model.push_to_hub(
 "your-repo-name/sample-name", use_auth_token=True
)

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:79: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'togethercomputer/Llama-2-7B-32K-Instruct' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Adding EOS to train dataset:   0%|          | 0/80 [00:00<?, ? examples/s]

AttributeError: 'NoneType' object has no attribute 'endswith'